In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from numpy import ma
from mayavi import mlab
from array import array

from vis_mayavi_tools import *

In [2]:
def read_3d_flow(fileName):

    print('Reading flow file: ', fileName)
    f = open(path + fileName, "rb")
    data = np.fromfile(f, dtype=np.float32)

    nx = int(data[0])
    ny = int(data[1])
    nz = int(data[2])

    # Exctract data (everything except dimensions)
    data = data[3:]

    size = 3*nx*ny*nz
    shape = (nz, ny, nx)

    print('Vector field dimensions:', nx, ny, nz )

    # Separate u and v components and reshape to the matrix
    u = data.take(np.arange(0,size,3)).reshape(shape)
    v = data.take(np.arange(1,size,3)).reshape(shape)
    w = data.take(np.arange(2,size,3)).reshape(shape)

    return u, v, w, shape

In [3]:
def read_3d_volume(fileName, nx, ny, nz):

    print('Reading 3D volume file: ', fileName)
    f = open(fileName, "rb")
    data = np.fromfile(f, dtype=np.float32)

    #size = nx*ny*nz
    shape = (nz, ny, nx)

    return data.reshape(shape)

In [4]:
def read_multi_tiff(path, n_images):
    """
    path - Path to the multipage-tiff file
    n_images - Number of pages in the tiff file
    """
    img = Image.open(path)
    images = []
    for i in range(n_images):
        try:
            img.seek(i)
            slice_ = np.zeros((img.height, img.width))
            
            #print slice_.shape
            
            for j in range(slice_.shape[0]):
                for k in range(slice_.shape[1]):
                    #print j,k
                    slice_[j,k] = img.getpixel((k, j))

            #print i
            images.append(slice_)

        except EOFError:
            # Not enough frames in img
            break

    return np.array(images)

In [5]:
nx = 700
ny = 600
nz = 500

path = 'y:\\tomo\\ershov\\bone_load\\syn13\\results_merged\\'

u = read_3d_volume(path + 'time07-vec_x-700-600-500.raw', nx, ny, nz)
v = read_3d_volume(path + 'time07-vec_y-700-600-500.raw', nx, ny, nz)
w = read_3d_volume(path + 'time07-vec_z-700-600-500.raw', nx, ny, nz)

#with Image.open(path + "\\mask_3_4.tif") as image_mask:
#    mask = np.array(image_mask)
#mask = read_multi_tiff(path + "\\mask_3_4.tif", nz)

    
#print(mask.shape)
print(u.shape)


Reading 3D volume file:  y:\tomo\ershov\bone_load\syn13\results_merged\time07-vec_x-700-600-500.raw
Reading 3D volume file:  y:\tomo\ershov\bone_load\syn13\results_merged\time07-vec_y-700-600-500.raw
Reading 3D volume file:  y:\tomo\ershov\bone_load\syn13\results_merged\time07-vec_z-700-600-500.raw
(500, 600, 700)


In [6]:
# Coordinates correction
u = np.flipud(u)
v = np.flipud(v)
w = np.flipud(w)

In [7]:
use_masking = False

In [8]:
if use_masking:    
    u = u*mask
    v = v*mask
    w = w*mask

In [9]:
    #x = np.arange(0, nx-1, 4)
    #y = np.arange(0, ny-1, 4)
    #z = np.arange(0, nz-1, 20)
    #
    #x2 = np.arange(0, nx-1, 5)
    #y2 = np.arange(85, ny-1, 85)
    #z2 = np.arange(0, nz-1, 5)
    #
#
    ##print x, y, z
    #xx, yy, zz = np.meshgrid(x, y, z)
    #xx2, yy2, zz2 = np.meshgrid(x2, y2, z2)
#
    ##print xx
#
    #print 'Original data size (z,y,x):', u.shape
#
    #pu = u[zz, yy, xx]
    #pv = v[zz, yy, xx]
    #pw = w[zz, yy, xx]
    #
    #pu2 = u[zz2, yy2, xx2]
    #pv2 = v[zz2, yy2, xx2]
    #pw2 = w[zz2, yy2, xx2]
    #
    
    # Visulaization 1: Manual
    # mlab.quiver3d(xx, yy, zz, pu, pv, pw, scale_factor=2.0, mode='cone')
    #mlab.quiver3d(xx2, yy2, zz2, pu2, pv2, pw2, scale_factor=2.0, mode='cone')

In [10]:
show_3d = True
if show_3d:
   
    # Visulaization 2: Mayavi
    
    u1 = np.swapaxes(u,0,2)
    v1 = np.swapaxes(v,0,2)
    w1 = np.swapaxes(w,0,2)
       
    #mask1 = np.swapaxes(mask,0,2)
    #mask1 = mask1[:,:,::-1]

    

    print('x-comp:', np.min(u1), np.max(u1))
    
    print('VTK data size (x,y,z):', u1.shape)
    
    data_size = u1.shape
    
    #nx = data_size[0]
    #ny = data_size[1]
    #nz = data_size[2]
    
    f = mlab.figure(size=(1200, 800))
    
    vec = mlab.pipeline.vector_field(u1, v1, w1)
    vec_amp = mlab.pipeline.extract_vector_norm(vec)
   
    
    #vec.scalar_data = curl1
    #vec.scalar_name = 'curl'
    #vec.update()
    
    
    #m = mlab.pipeline.scalar_field(mask1)       # Mask labels
    #mlab.pipeline.iso_surface(m, contours=[1.0], color=(0,0.3,1.0), opacity=0.1)
    
    # Vis mode
    #mlab.options.backend = 'envisage'
    
    
    
    # Viz: Vector cut plane
    #mlab.pipeline.vector_cut_plane(vec, scale_factor=8, mask_points=10, mode='cone', plane_orientation='z_axes')
    #vector_cut_plane = engine.scenes[0].children[0].children[0].children[0]
    
    
    # Camera view settings
    # Front view
    mlab.view(-90, 150)
    
    
    # Isometric view
    #f = mlab.gcf()
    #camera = f.scene.camera
    #camera.position = [286.23096991964076, -143.15658886894965, -467.81898197813479]
    #camera.focal_point = [100.49999809265137, 85.499998092651367, 35.500003814697266]
    #camera.view_angle = 30.0
    #camera.view_up = [-0.10467072871467387, -0.91942928835552484, 0.3790696799592288]
    #camera.clipping_range = [254.92021693034593, 1026.1163574621135]
    #camera.compute_view_plane_normal()
    #
    
    # Frontal view
    #camera = f.scene.camera
    #camera.position = [609.41770113840994, -29.144106009077596, 38.941433671301574]
    #camera.focal_point = [100.5, 85.5, 35.5]
    #camera.view_angle = 30.0
    #camera.view_up = [-0.21980780092670132, -0.9755102894573745, 0.0080128530846999176]
    #camera.clipping_range = [285.8987915866569, 819.74331199974517]
    #camera.compute_view_plane_normal()
#
  
    #mlab.pipeline.iso_surface(vorticity, contours=[0.1, 0.3, 0.5], opacity=0.5)
    #mlab.pipeline.iso_surface(curl, contours=[0.1, 0.3, 0.5], opacity=0.5)
    #mlab.pipeline.volume(curl)
    
    
    mlab.orientation_axes()
    #print f.scene.children[0].children[0].children[0]
    engine = mlab.get_engine()
        
    
    # Disable widget controls
    #vector_cut_plane.implicit_plane.widget.enabled = False
    
    from mayavi.modules.outline import Outline
    outline1 = Outline()
    array_source = engine.scenes[0].children[0]
    engine.add_filter(outline1, array_source)
    
    e = mlab.pipeline.extract_vector_components(vec)
    e.component = 'x-component'
    
    streamline = mlab.pipeline.streamline(e, seedtype='plane',
                                        integration_direction='both',
                                        colormap='jet')
    
    streamline.seed.widget.resolution = 15
    
    
    
    
    #print mlab.pipeline.streamline()
    
    #streamline.module_manager.source = curl_amp 
    
    #set_yz_side_cut_plane(streamline, data_size, 170)
    set_xy_front_cut_plane(streamline, data_size, 35)

    
    
    
    #print streamline.seed.update_poly_data()
    
    refresh_widget(streamline)
    
    
    mlab.colorbar(streamline, title='x-comp', orientation='vertical', nb_labels=7)
    
    
    #streamline.update_streamlines = 0
    
    #mlab.view(-90, 150)
    
    #streamline.seed.widget.update_traits())
    
    #print range(0, nx, 10)
    
    # Make animation
    if False:
        for i in range(5, nz, 5):
            
            # Viz 1: Vector field
            #vector_cut_plane.implicit_plane.plane.origin = np.array([ 100.5       ,   85.5       ,   10*(i+1)])
            
            set_xy_front_cut_plane(streamline, data_size, i)
            
            refresh(streamline)
    
            # Viz 2: Streamlines
            mlab.savefig(path + 'figures\\' + 'fig' +str(i).zfill(3) + '.png')
    
   
    
    #vector_cut_plane = engine.scenes[0].children[0].children[0].children[0]
    
    
    mlab.show()
    
  
    #vector_cut_plane = f.scene.children[0].children[0].children[0]
    

    #vector_cut_plane.implicit_plane.plane.origin = array('d', [ 100.5       ,   85.5       ,   70])
    
   
    
    

x-comp: -10.33418 10.840409
VTK data size (x,y,z): (700, 600, 500)
